In [2]:
from pose_graph import PoseGraph
import numpy as np

In [ ]:
sensor_id = ["cam0", "cam1", "imu0", "imu1"]

pose_id = ["cam1", "cam2", "cam3", "cam4", "cam5"]

poses = {
    "cam1": [[1, "M"], [2, "N"], [3, "O"], [4, "P"], [5, "Q"]],
    "cam2": [(2, "N")],
    "cam3": [(3, "O")],
    "cam4": [(4, "P")],
    "cam5": [(5, "Q")],
}

# for pose in poses.items():
#     print(pose[1][1])

if 1 not in poses["cam1"]:
    print("not in")

print(poses["cam1"])


In [ ]:
edge = {"sensor": {"cam0": 1, "cam1": 2, "imu0": 3, "imu1": 4}}

# if "sensor" in edge:
#     print("yes")

# else:
#     print("no") 

# try:
#     edge["sensor"]["cam2"]
#     print("yes")
# except KeyError:
#     print("no")

if "cam0" in edge.get("sensor",{}):
    print("yes")


str1 = "cam0"

list1 = ["cam0", "cam1", "cam2", "cam3", "cam4", "cam5"]
newstr = "cam6"+str1
list1.append("cam6"+str1)
print(list1)

In [1]:
from pose_graph import PoseGraph
import numpy as np
pg = PoseGraph()

sensor_id = "1"

tf11 = np.array(
    [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf12 = np.array(
    [
        [1, 0, 0, 1],
        [0, 1, 0, 2],
        [0, 0, 1, 3],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf13 = np.array(
    [
        [1, 0,	0,	10],
        [0,0.707106781186548,	-0.707106781186548,	2],
        [0,0.707106781186548,	0.707106781186548,	10],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf14 = np.array(
    [
        [0, -0.866025403784439,	0.500000000000000,	0],
        [1,     0,	0,	0],
        [0, 0.5,	0.866025403784439,	-5],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

pg.add_sensor(
    sensor_id,
    {
        "1": [tf11, True],
        "2": [tf12, True],
        "3": [tf13, True],
        # "4": [tf14, True],
    },
)

sensor_id = "5"

tf52 = np.array(
    [
        [1, 0, 0, 1],
        [0, 0, 1, -17],
        [0, -1, 0, 8],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf53 = np.array(
    [
        [1,	0,	0,	10],
        [0,	0.707106781186548,	0.707106781186548,	-10],
        [0,	-0.707106781186548,	0.707106781186548,	8],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

tf54 = np.array(
    [
        [0, -0.866025403784439,	0.500000000000000, 0],
        [0, 0.500000000000000,	0.866025403784439,	-25],
        [-1, 0, 0, 10],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

pg.add_sensor(
    sensor_id,
    {
        "2": [tf52, True],
        "3": [tf53, True],
        "4": [tf54, True],
    },
)

pg.update_graph(
    "1",
    {
        "2": [tf12, False],
        "3": [tf13, True],
        "4": [tf14, True],
    },
)

tf14 = pg.get_transform(parent_id="1", child_id="1", solver_method="BFS")

print(tf14)

# print(pg.object_id)
# print(pg.edges)

The object is not in the graph, add it to the graph ... 
The edge between the parent node and the child node is active.
 Directly return the transformation
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


### Test the integration of posegraph and communication functions


In [ ]:
import posehub_main
import sys
import argparse
from pose_graph import PoseGraph

pose_graph = PoseGraph()

args = argparse.Namespace(    
    sub_ip="192.168.1.23",
    sub_port="5588",
    pub_port="5589",
    sub_topic=[b"tool_1", b"tool_2", b"tool_3"],
    pub_topic=[b"topic4", b"topic5", b"topic6"],
    sensor_name="h1"
    )


zmq_manager_1, sub_thread_1, pub_thread_1 = posehub_main.initialize_ZMQManager(
        args.sub_ip,
        args.sub_port,
        args.pub_port,
        args.sub_topic,
        args.pub_topic,
        args.sensor_name,
    )
    # zmq_manager_2, sub_thread_2, pub_thread_2 = initialize_ZMQManager(args.sub_ip, args.sub_port, args.pub_port, args.sub_topic, args.pub_topic)
i = 0
try:
    while True:
        # Running the main loop

        # test receiving poses
        # print('tool 1: ', zmq_manager_1.sub_poses['tool_1'])
        poseinfo = posehub_main.receive_poses(args, zmq_manager_1)
        if poseinfo is not None:
            print(poseinfo)
        else:
            print("no poseinfo received")
        # print('tool 2: ', zmq_manager_1.sub_poses['tool_2'])
        # print('tool 3: ', zmq_manager_1.sub_poses['tool_3'])

        # test sending messages
        zmq_manager_1.pub_messages["topic4"] = f"topic4 test message {i}"
        zmq_manager_1.pub_messages["topic5"] = f"topic5 test message {i}"
        zmq_manager_1.pub_messages["topic6"] = f"topic6 test message {i}"
        i += 1e-6

except KeyboardInterrupt:
    posehub_main.terminate_ZMQManager(zmq_manager_1, sub_thread_1, pub_thread_1)
    # terminate_ZMQManager(zmq_manager_2, sub_thread_2, pub_thread_2)


